In [1]:
# %matplotlib ipympl
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *

In [4]:
dgilib_path = "C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Python\\dgilib.dll"

In [5]:
fig = plt.figure(figsize=(8, 6))
fig.show()

In [6]:
data = []

config_dict = {
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "write_mode": [False, False, False, False],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "gpio_delay_time" : 0.010795,
    "fig": fig,
    "verbose": 0,
}

fig.clf()

with DGILibExtra(dgilib_path, **config_dict) as dgilib:
    dgilib.device_reset()
    data = dgilib.logger(1)
#     dgilib.ax.plot(*dgilib.power_filter_by_pin(2))
#     dgilib.ax.plot(*dgilib.power_filter_by_pin(3))

In [7]:
data[INTERFACE_GPIO]

([0.0013472,
  0.009144533333333333,
  0.12989973333333332,
  0.13998026666666666,
  0.2607354666666667,
  0.27081653333333333,
  0.391568,
  0.40164853333333334,
  0.5224069333333333,
  0.5324885333333333,
  0.6532501333333334,
  0.6633434666666667,
  0.7840885333333334,
  0.7941696,
  0.9149168,
  0.9249968000000001],
 [[True, True, False, False],
  [True, True, False, False],
  [True, True, True, False],
  [True, True, True, False],
  [True, True, False, False],
  [True, True, False, False],
  [True, True, True, False],
  [True, True, True, False],
  [True, True, False, False],
  [True, True, False, False],
  [True, True, True, False],
  [True, True, True, False],
  [True, True, False, False],
  [True, True, False, False],
  [True, True, True, False],
  [True, True, True, False]])

In [8]:
args = [[INTERFACE_GPIO, INTERFACE_POWER]]

In [10]:
args[0]

[48, 256]

In [14]:
dict({INTERFACE_POWER: ([], []), INTERFACE_GPIO: ([], [])})

In [30]:
attr_dict = {
    "spi": INTERFACE_SPI,
    "usart": INTERFACE_USART,
    "i2c": INTERFACE_I2C,
    "gpio": INTERFACE_GPIO,
    "power": INTERFACE_POWER,
}

In [501]:
class LoggerData(dict):
    """Class to store DGILib Logger Data."""

    __slots__ = attr_dict.values()

    def __init__(self, *args, **kwargs):
        """Take list of interfaces for the data."""
        # Call init function of dict
        super().__init__(self)
        # No args or kwargs were specified, populate args[0] with standard
        # interfaces
        if not args and not kwargs:
            args = [[INTERFACE_GPIO, INTERFACE_POWER]]
        # Args is list of interfaces, make data dict with interfaces as keys
        # and tuples of lists as values
        if args and isinstance(args[0], list):
            for interface in args[0]:
                self[interface] = ([], [])
        # Instantiate dict with arguments
        else:
            self.update(*args, **kwargs)
            assert all(valid_tuple(tup) for tup in self.values()), f"One of the tuples passed to LoggerData was invalid. {self.values()}"

    def __iadd__(self, loggerdata):
        """Append new data.

        Used to provide `loggerdata1 += loggerdata` syntax
        """
        if not isinstance(loggerdata, dict):
            raise ValueError(
                f"loggerdata must be a dict or LoggerData. Got {type(loggerdata)}")
        for interface, values in loggerdata.items():
            if interface in self.keys():
                self[interface][0].extend(values[0])
                self[interface][1].extend(values[1])
            else:
                self[interface] = values
        return self

    def __add__(self, loggerdata):
        """Append new data.

        Used to provide `loggerdata2 = loggerdata1 + loggerdata` syntax
        """
        data = LoggerData()
        data += self
        data += loggerdata
        return data

    def append(self, interface, sample):
        """Append a sample to one of the interfaces."""
        pass

    def extend(self, interface, samples):
        """Append a list of samples to one of the interfaces."""
        pass

    def length(self, attr=None):
        """Compute the number of samples for the interfaces.

        Return a dict of the number of samples for each interface. Or the
        length of the `attr` specified.
        """
        if attr is None:
            return {key: len(val[0]) for key, val in self.items()}
        elif attr in self.keys():
            return len(self[attr][0])
        elif attr in attr_dict.keys():
            return len(self[attr_dict[attr]][0])
        else:
            raise ValueError(
                f"attr must be a named or numbered interface. Got {attr}")

    def __getattr__(self, attr):
        """Get attribute.

        Used to provide `data.spi` syntax.
        """
        return self[attr_dict[attr]]

    def __setattr__(self, attr, value):
        """Set attribute.

        Used to provide `data.spi = tuple([], [])` syntax.
        """
        self[attr_dict[attr]] = value

    def __repr__(self):
        """Print data.

        Used to provide `str(data)` syntax.
        """
        output = "Interfaces:\n"
        for interface in self.keys():
            if interface in attr_dict.values():
                for name, number in attr_dict.items():
                    if interface == number:
                        output += f"\t{number:4}: {name:{' '}^{10}}, samples: {len(self[interface][0]):7}\n"
                        break
            else:
                output += f"\t{interface:4}: (unknown) , samples: {len(self[interface][0]):7}\n"

        return output

TypeError: __slots__ items must be strings, not 'int'

In [458]:
def valid_tuple(tup):
    """Check if tuple is valid data tuple for interface."""
    return isinstance(tup, tuple) and len(tup) == 2 and all(isinstance(s, list) for s in tup) and len(tup[0]) == len(tup[1]) 

In [459]:
valid_tuple(([1], [2]))

True

In [460]:
def test_init_from_dict():
    data = LoggerData(
        {INTERFACE_POWER: ([], []), INTERFACE_GPIO: ([], []), 4: ([], [])})
    print(data)
    assert data[INTERFACE_POWER] == ([], [])
    assert data[INTERFACE_GPIO] == ([], [])

In [461]:
test_init_from_dict()

Interfaces:
	 256:   power   , samples:       0
	  48:    gpio   , samples:       0
	   4: (unknown) , samples:       0



In [462]:
def test__iadd__():
    data = LoggerData({INTERFACE_POWER: ([1], [2])})
    data += {INTERFACE_POWER: ([2], [3])}
    assert data[INTERFACE_POWER] == ([1, 2], [2, 3])
    data = LoggerData({INTERFACE_POWER: ([1], [2])})
    data += LoggerData({INTERFACE_POWER: ([2], [3])})
    assert data[INTERFACE_POWER] == ([1, 2], [2, 3])

def test__add__():
    data1 = LoggerData({INTERFACE_POWER: ([1], [2])})
    data2 = LoggerData({INTERFACE_POWER: ([2], [3])})
    data = data1 + data2

In [463]:
test__iadd__()

In [464]:
data = LoggerData({INTERFACE_POWER: ([1], [2])})

In [465]:
data += {INTERFACE_POWER: ([2], [3])}

In [466]:
data.power

([1, 2], [2, 3])

In [467]:
data = LoggerData({INTERFACE_POWER: ([1], [2])})
data += LoggerData({INTERFACE_POWER: ([2], [3])})

In [468]:
data.power

([1, 2], [2, 3])

In [469]:
test__add__()

In [470]:
data1 = LoggerData({INTERFACE_POWER: ([1], [2])})
data2 = LoggerData({INTERFACE_POWER: ([2], [3])})
data = data1 + data2

In [471]:
data.power

([1, 2], [2, 3])

In [472]:
data1.power

([1], [2])

In [473]:
data2.power

([2], [3])

In [474]:
del data1
del data2

In [475]:
data.power

([1, 2], [2, 3])

In [476]:
data1 = LoggerData({INTERFACE_POWER: ([1], [2])})
data2 = LoggerData({INTERFACE_POWER: ([2], [3])})
data = data1 + data2
assert data[INTERFACE_POWER] == ([1, 2], [2, 3])
data1[INTERFACE_POWER] = ([4], [5])
assert data[INTERFACE_POWER] == ([1, 2], [2, 3])

In [477]:
data.power

([1, 2], [2, 3])

In [478]:
data = LoggerData({INTERFACE_POWER: ([1], [2]), INTERFACE_GPIO: ([], []), 4: ([3,4], [5,6])})

In [479]:
data = LoggerData({INTERFACE_POWER: ([1], [2])})
data1 = data + {}
data[INTERFACE_POWER] = ([4], [5])
data1[INTERFACE_POWER]

([1], [2])

In [480]:
LoggerData({INTERFACE_POWER: ([], [2]), INTERFACE_GPIO: ([], []), 4: ([3,4], [5,6])})

AssertionError: One of the tuples passed to LoggerData was invalid. dict_values([([], [2]), ([], []), ([3, 4], [5, 6])])

In [481]:
# Simple addition of objects
data1 = LoggerData({INTERFACE_POWER: ([1], [2])})
data2 = LoggerData({INTERFACE_POWER: ([2], [3])})
data = data1 + data2
assert data[INTERFACE_POWER] == ([1, 2], [2, 3]), "Incorrect value"

In [482]:
data.power

([1, 2], [2, 3])

In [484]:
# Check that data has been deep copied
# data1[INTERFACE_POWER]
assert data[INTERFACE_POWER] == ([1, 2], [2, 3]), "Incorrect value"
# Delete original copies (decrease reference count to them)
del data1
del data2
assert data[INTERFACE_POWER] == ([1, 2], [2, 3]), "Incorrect value"

In [486]:
data = LoggerData({INTERFACE_POWER: ([1], [2])}), "Incorrect value"
data1 = data
# data[INTERFACE_POWER] = ([4], [5])
assert data1[INTERFACE_POWER] == ([4], [5]), "Incorrect value"
# Check that data has been deep copied
data = LoggerData({INTERFACE_POWER: ([1], [2])})
data1 = data + {}
data[INTERFACE_POWER] = ([4], [5])
assert data1[INTERFACE_POWER] == ([1], [2]), "Incorrect value"

IndexError: tuple index out of range

In [451]:
len(data)

2

In [221]:
data.length(INTERFACE_GPIO)

256 ([1], [2]) 1
48 ([], []) 0
4 ([], []) 0


0

In [222]:
data.length(INTERFACE_POWER)

256 ([1], [2]) 1
48 ([], []) 0
4 ([], []) 0


1

In [226]:
data.length(4)

256 ([1], [2]) 1
48 ([], []) 0
4 ([], []) 0


0

In [224]:
data.length()

256 ([1], [2]) 1
48 ([], []) 0
4 ([], []) 0


{256: 1, 48: 0, 4: 0}

In [272]:
isinstance({256: 1, 48: 0, 4: 0}, LoggerData)

False

In [605]:
class InterfaceData(tuple):
    """Class to store DGILib Logger Interface Data."""
    
    def __new__(*args, **kwargs):
        """Take tuple of timestamps and values."""
        # Call init function of tuple
        if len(args) == 1:
            args = *args, ([], [])
        assert valid_interface_data(args[1]), f"Argument passed to InterfaceData was invalid. {args[1]}"
        return tuple.__new__(*args, **kwargs)
    
    @property
    def timestamps(self):
        return self[0]
    
    @property
    def values(self):
        return self[1]
    
    def __len__(self):
        return len(self[0])
    
def valid_interface_data(samples):
    """Check if samples are valid InterfaceData."""
    return (isinstance(samples, (tuple, list)) and
            len(samples) == 2 and
            all(isinstance(sample, list) for sample in samples) and
            len(samples[0]) == len(samples[1]))

In [606]:
q = InterfaceData([[2],[3]])
q

(<class '__main__.InterfaceData'>, [[2], [3]])
(<class '__main__.InterfaceData'>, [[2], [3]])


([2], [3])

In [607]:
q = InterfaceData()
q

(<class '__main__.InterfaceData'>,)
(<class '__main__.InterfaceData'>, ([], []))


([], [])

In [590]:
q[1]

[3]

In [572]:
q.timestamps

[2]

In [531]:
w = tuple([[2],[3, 3]])
w

([2], [3, 3])

In [511]:
e = [4]

In [515]:
w[1].extend(e)

In [516]:
w

([2], [3, [4], 4])

In [539]:
isinstance((), MutableSequence)

NameError: name 'MutableSequence' is not defined

In [664]:

attr_dict = {
    "spi": INTERFACE_SPI,
    "usart": INTERFACE_USART,
    "i2c": INTERFACE_I2C,
    "gpio": INTERFACE_GPIO,
    "power": INTERFACE_POWER,
}


class InterfaceData(tuple):
    """Class to store DGILib Logger Interface Data."""

    def __new__(*args, **kwargs):
        """Take tuple of timestamps and values."""
        # Call init function of tuple
        if len(args) == 1:
            args = *args, ([], [])
        assert valid_interface_data(
            args[1]), f"Samples passed to InterfaceData were not valid_interface_data. {args[1]}"
        return tuple.__new__(*args, **kwargs)

    @property
    def timestamps(self):
        """Get the list of timestamps."""
        return self[0]

    @property
    def values(self):
        """Get the list of values."""
        return self[1]

    def __iadd__(self, interface_data):
        """Append new interface_data (in-place).

        Used to provide `interface_data += interface_data1` syntax
        """
        if not isinstance(interface_data, InterfaceData):
            assert valid_interface_data(
                interface_data), f"Samples passed to InterfaceData were not valid_interface_data. {interface_data}"
        self[0].extend(interface_data[0])
        self[1].extend(interface_data[1])
        return self

    def __add__(self, interface_data):
        """Append new interface_data (copy).

        Used to provide `interface_data2 = interface_data1 + interface_data` syntax
        """
        data = InterfaceData()
        data += self
        data += interface_data
        return data

    def append(self, interface_data):
        """Append interface_data."""
        return self.extend(interface_data)

    def extend(self, interface_data):
        """Append a list of interface_data."""
        if isinstance(interface_data, InterfaceData):
            self += interface_data
        else:
            self += InterfaceData(interface_data)
        return self

    def __len__(self):
        """Get the number of samples."""
        return len(self[0])


class LoggerData(dict):
    """Class to store DGILib Logger Data."""

    def __init__(self, *args, **kwargs):
        """Take list of interfaces for the data."""
        # Call init function of dict
        super().__init__(self)
        # No args or kwargs were specified, populate args[0] with standard
        # interfaces
        if not args and not kwargs:
            args = [[INTERFACE_GPIO, INTERFACE_POWER]]
        # Args is list of interfaces, make data dict with interfaces as keys
        # and tuples of lists as values
        if args and isinstance(args[0], list):
            for interface in args[0]:
                self[interface] = InterfaceData()
        # Instantiate dict with arguments
        else:
            self.update(*args, **kwargs)

        for interface, interface_data in self.items():
            if not isinstance(interface_data, InterfaceData):
                self[interface] = InterfaceData(interface_data)

    def __iadd__(self, logger_data):
        """Append new logger_data (in-place).

        Used to provide `logger_data1 += logger_data` syntax
        """
        if not isinstance(logger_data, dict):
            raise ValueError(
                f"logger_data must be a dict or LoggerData. Got {type(logger_data)}")
        for interface, interface_data in logger_data.items():
            self.extend(interface, interface_data)
        return self

    def __add__(self, logger_data):
        """Append new logger_data (copy).

        Used to provide `logger_data2 = logger_data1 + logger_data` syntax
        """
        data = LoggerData()
        data += self
        data += logger_data
        return data

    def append(self, interface, interface_data):
        """Append a sample to one of the interfaces."""
        return self.extend(interface, interface_data)

    def extend(self, interface, interface_data):
        """Append a list of samples to one of the interfaces."""
        if interface in self.keys():
            self[interface].extend(interface_data)
        elif not isinstance(interface_data, InterfaceData):
            self[interface] = InterfaceData(interface_data)
        else:
            self[interface] = interface_data
        return self

    def length(self, attr=None):
        """Compute the number of samples for the interfaces.

        Return a dict of the number of samples for each interface. Or the
        length of the `attr` specified.
        """
        if attr is None:
            return {key: len(interface_data) for key, interface_data in self.items()}
        elif attr in self.keys():
            return len(self[attr])
        elif attr in attr_dict.keys():
            return len(self[attr_dict[attr]])
        else:
            raise ValueError(
                f"attr must be a named or numbered interface. Got {attr}")

    def __getattr__(self, attr):
        """Get attribute.

        Used to provide `data.spi` syntax.
        """
        return self[attr_dict[attr]]

    def __setattr__(self, attr, value):
        """Set attribute.

        Used to provide `data.spi = tuple([], [])` syntax.
        """
        self[attr_dict[attr]] = value

    def __repr__(self):
        """Print data.

        Used to provide `str(data)` syntax.
        """
        output = "Interfaces:\n"
        for interface in self.keys():
            if interface in attr_dict.values():
                for name, number in attr_dict.items():
                    if interface == number:
                        output += f"\t{number:4}: {name:{' '}^{10}}, samples: {len(self[interface][0]):7}\n"
                        break
            else:
                output += f"\t{interface:4}: (unknown) , samples: {len(self[interface][0]):7}\n"

        return output

# # Load CSV

# # Calculations


def valid_interface_data(samples):
    """Check if samples are valid InterfaceData."""
    return (isinstance(samples, (tuple, list)) and
            len(samples) == 2 and
            all(isinstance(sample, list) for sample in samples) and
            len(samples[0]) == len(samples[1]))


In [665]:
data = InterfaceData([[1], [2]])
data += InterfaceData([[2], [3]])

In [669]:
data

([1, 2], [2, 3])

In [681]:
data = LoggerData([INTERFACE_GPIO, INTERFACE_POWER])
data += LoggerData({
            INTERFACE_POWER: InterfaceData(([1], [2])),
            INTERFACE_GPIO: InterfaceData(([3], [4]))})
data += LoggerData({
            INTERFACE_POWER: InterfaceData(([1], [2])),
            INTERFACE_GPIO: InterfaceData(([3], [4]))})
data += LoggerData({
            INTERFACE_POWER: InterfaceData(([1], [2])),
            INTERFACE_GPIO: InterfaceData(([3], [4]))})

In [684]:
data.extend(INTERFACE_POWER, InterfaceData(([1], [2])))

Interfaces:
	  48:    gpio   , samples:       3
	 256:   power   , samples:       4

In [666]:
b = InterfaceData()

In [667]:
isinstance(a, InterfaceData)

False

In [668]:
isinstance((), InterfaceData)

False

In [650]:
# Simple instantiaton
data = LoggerData()
assert data[INTERFACE_POWER] == ([], []), "Incorrect value"
assert data[INTERFACE_GPIO] == ([], []), "Incorrect value"

In [651]:
# Instantiaton from list of interfaces
data = LoggerData([INTERFACE_GPIO, INTERFACE_POWER])
assert data[INTERFACE_POWER] == ([], []), "Incorrect value"
assert data[INTERFACE_GPIO] == ([], []), "Incorrect value"

In [652]:
# Instantiation from dictionary
data = LoggerData(
    {INTERFACE_POWER: ([], []), INTERFACE_GPIO: ([], [])})
print(data)
assert data[INTERFACE_POWER] == ([], []), "Incorrect value"
assert data[INTERFACE_GPIO] == ([], []), "Incorrect value"

Interfaces:
	 256:   power   , samples:       0
	  48:    gpio   , samples:       0



In [653]:
# Instantiation from dictionary with data
data = LoggerData(
    {INTERFACE_POWER: ([1], [2]), INTERFACE_GPIO: ([3], [4])})
assert data[INTERFACE_POWER] == ([1], [2]), "Incorrect value"
assert data[INTERFACE_GPIO] == ([3], [4]), "Incorrect value"

In [654]:
# Simple instantiaton
data = LoggerData()
assert data[INTERFACE_POWER] == ([], []), "Incorrect value"
assert data[INTERFACE_GPIO] == ([], []), "Incorrect value"

In [655]:
data = LoggerData([2])

In [685]:
type(data[2])

KeyError: 2

In [ ]:
class testest(dict):
    """Class to store DGILib Logger Data."""

    def __init__(self, *args, **kwargs):
        """Take list of interfaces for the data."""
        # Call init function of dict
        super().__init__(self)
        self.update(*args, **kwargs)

In [ ]:
testest()

In [ ]:
isinstance(LoggerData, dict)

In [686]:
l = [1,2]

In [687]:
t = (2,3)

In [688]:
a,b = l

In [689]:
a

1

In [690]:
b

2

In [691]:
c,d = t

In [692]:
c

2

In [693]:
d

3

In [1970]:
"""Class to store DGILib Logger Data."""

from pydgilib_extra.dgilib_extra_config import (
    INTERFACE_SPI, INTERFACE_USART, INTERFACE_I2C, INTERFACE_GPIO,
    INTERFACE_POWER)

attr_dict = {
    "spi": INTERFACE_SPI,
    "usart": INTERFACE_USART,
    "i2c": INTERFACE_I2C,
    "gpio": INTERFACE_GPIO,
    "power": INTERFACE_POWER,
}


class InterfaceData(object):
    """Class to store DGILib Logger Interface Data."""

    __slots__ = ['timestamps', 'values']

    def __init__(self, *args):
        """Take tuple of timestamps and values."""
        if not args:
            self.timestamps = []
            self.values = []
        elif len(args) == 1 and isinstance(args[0], InterfaceData):
            self = args[0]
        elif len(args) == 1 and valid_interface_data(args[0]):
            self.timestamps, self.values = args[0]
        elif len(args) == 2 and isinstance(args[0], list) and isinstance(args[1], list):
            self.timestamps, self.values = args
        elif len(args) == 2 and isinstance(args[0], (int, float)) and isinstance(args[1], (int, float, list)):
            self.timestamps = [args[0]]
            self.values = [args[1]]
        else:
            raise ValueError(
                f"Samples passed to InterfaceData must be tuple([],[]) or timestamps, values or InterfaceData. Got {args}")

    def __iadd__(self, interface_data):
        """Append new interface_data (in-place).

        Used to provide `interface_data += interface_data1` syntax
        """
        if isinstance(interface_data, InterfaceData):
            self.timestamps.extend(interface_data.timestamps)
            self.values.extend(interface_data.values)
        else:
            assert valid_interface_data(
                interface_data), f"Samples passed to InterfaceData were not valid_interface_data. {interface_data}"
            self.timestamps.extend(interface_data[0])
            self.values.extend(interface_data[1])
        return self

    def __add__(self, interface_data):
        """Append new interface_data (copy).

        Used to provide `interface_data2 = interface_data1 + interface_data` syntax
        """
        data = InterfaceData()
        data += self
        data += interface_data
        return data

    def append(self, interface_data):
        """Append interface_data."""
        return self.extend(interface_data)

    def extend(self, interface_data):
        """Append a list of interface_data."""
        if isinstance(interface_data, InterfaceData):
            self += interface_data
        else:
            self += InterfaceData(interface_data)
        return self

    def __len__(self):
        """Get the number of samples."""
        if len(self.timestamps):
            return len(self.timestamps)
        else:
            return 0

    def __getitem__(self, index):
        """Get item.

        Used to provide `timestamp, value = interface_data[5]` and
        `timestamp, value = interface_data[2:5]` syntax
        """
        return (self.timestamps[index], self.values[index])
        # # Provides data["timestamps"] as well
        # if index == self.__slots__[0]:
        #     return self.timestamps
        # elif index == self.__slots__[1]:
        #     return self.values
        # else:
        #     return (self.timestamps[index], self.values[index])

    def __contains__(self, item):
        """Contains.

        Used to provide `([1], [2]) in interface_data` syntax
        """
        if isinstance(item, InterfaceData):
            return all(any(item_timestamp == self_timestamp and item_value == self_value for self_timestamp, self_value in self) for item_timestamp, item_value in item)
        else:
            _item = InterfaceData(item)
            return all(any(item_timestamp == self_timestamp and item_value == self_value for self_timestamp, self_value in self) for item_timestamp, item_value in _item)


class LoggerData(dict):
    """Class to store DGILib Logger Data."""

    def __init__(self, *args, **kwargs):
        """Take list of interfaces for the data."""
        # Call init function of dict
        super().__init__(self)
        # No args or kwargs were specified, populate args[0] with standard
        # interfaces
        if not args and not kwargs:
            args = [[INTERFACE_GPIO, INTERFACE_POWER]]
        # Args is list of interfaces, make data dict with interfaces as keys
        # and tuples of lists as values
        if args and isinstance(args[0], list):
            for interface in args[0]:
                self[interface] = InterfaceData()
        # Instantiate dict with arguments
        else:
            self.update(*args, **kwargs)

        for interface, interface_data in self.items():
            if not isinstance(interface_data, InterfaceData):
                self[interface] = InterfaceData(interface_data)

    def __getattr__(self, attr):
        """Get attribute.

        Used to provide `data.spi` syntax.
        """
        return self[attr_dict[attr]]

    def __setattr__(self, attr, value):
        """Set attribute.

        Used to provide `data.spi = InterfaceData(([], []))` syntax.
        """
        self[attr_dict[attr]] = value

    def __iadd__(self, logger_data):
        """Append new logger_data (in-place).

        Used to provide `logger_data1 += logger_data` syntax
        """
        if not isinstance(logger_data, dict):
            raise ValueError(
                f"logger_data must be a dict or LoggerData. Got {type(logger_data)}")
        for interface, interface_data in logger_data.items():
            self.extend(interface, interface_data)
        return self

    def __add__(self, logger_data):
        """Append new logger_data (copy).

        Used to provide `logger_data2 = logger_data1 + logger_data` syntax
        """
        data = LoggerData()
        data += self
        data += logger_data
        return data

    # def __copy__(self):
    #     return self

    def append(self, interface, interface_data):
        """Append a sample to one of the interfaces."""
        return self.extend(interface, interface_data)

    def extend(self, interface, interface_data):
        """Append a list of samples to one of the interfaces."""
        if interface in self.keys():
            self[interface].extend(interface_data)
        elif not isinstance(interface_data, InterfaceData):
            self[interface] = InterfaceData(interface_data)
        else:
            self[interface] = interface_data
        return self

    def length(self, attr=None):
        """Compute the number of samples for the interfaces.

        Return a dict of the number of samples for each interface. Or the
        length of the `attr` specified.
        """
        if attr is None:
            return {key: len(interface_data) for key, interface_data in self.items()}
        elif attr in self.keys():
            return len(self[attr])
        elif attr in attr_dict.keys():
            return len(self[attr_dict[attr]])
        else:
            raise ValueError(
                f"attr must be a named or numbered interface. Got {attr}")

    def __str__(self):
        """Print data.

        Used to provide `str(data)` syntax.
        """
        output = "Interfaces:\n"
        for interface in self.keys():
            if interface in attr_dict.values():
                for name, number in attr_dict.items():
                    if interface == number:
                        output += f"\t{number:4}: {name:{' '}^{10}}, samples: {len(self[interface][0]):7}\n"
                        break
            else:
                output += f"\t{interface:4}: (unknown) , samples: {len(self[interface][0]):7}\n"

        return output

# # Load CSV

# # Calculations


def valid_interface_data(samples):
    """Check if samples are valid InterfaceData."""
    return (isinstance(samples, (tuple, list)) and
            len(samples) == 2 and
            all(isinstance(sample, list) for sample in samples) and
            len(samples[0]) == len(samples[1]))


In [1971]:
a = InterfaceData()
len(a)

0

In [1972]:
tuple(a)

()

In [1973]:
b = InterfaceData(([1],[3]))

In [1974]:
tuple(b)

((1, 3),)

In [1975]:
list(b)

[(1, 3)]

In [1976]:
c = InterfaceData(([1, 2],[3,4]))

In [1977]:
tuple(c)

((1, 3), (2, 4))

In [1978]:
list(c)

[(1, 3), (2, 4)]

In [1979]:
list(list(reversed(c)))

[(2, 4), (1, 3)]

In [1980]:
len(c)

2

In [1981]:
aa, bb = c

In [1982]:
aa

(1, 3)

In [1983]:
bb

D:\Anaconda3\envs\SAML11-32\lib\site-packages\IPython\core\displayhook.py:271: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  'Flushing oldest {cull_count} entries.'.format(sz=sz, cull_count=cull_count))


(2, 4)

In [1984]:
cc = c

In [1985]:
cc

In [1986]:
bool(a)

False

In [1987]:
bool(c)

True

In [1988]:
c.timestamps

[1, 2]

In [1989]:
c.values

[3, 4]

In [1990]:
c["timestamps"]

TypeError: list indices must be integers or slices, not str

In [1991]:
c[0:1]

([1], [3])

In [1992]:
d = InterfaceData(([1, 2, 3],[4, 5, 6]))

In [1993]:
[print(s) for s in reversed(d)]

(3, 6)
(2, 5)
(1, 4)


[None, None, None]

In [1994]:
e = reversed(d)

In [1995]:
e[0]

TypeError: 'reversed' object is not subscriptable

In [1996]:
tuple(InterfaceData([[1], [2]]))

((1, 2),)

In [1997]:
data = InterfaceData([[1], [2]])

In [1998]:
data.timestamps

[1]

In [1999]:
data.values

[2]

In [2000]:
assert data.timestamps == [1], "Incorrect value"

In [2001]:
assert data.values == [2], "Incorrect value"

In [2002]:
        data = InterfaceData([[1], [2]])
        assert data.timestamps == [1], "Incorrect value"
        assert data.values == [2], "Incorrect value"
        assert data["timestamps"] == [1], "Incorrect value"
        assert data["values"] == [2], "Incorrect value"

TypeError: list indices must be integers or slices, not str

In [2003]:
data = InterfaceData(([1, 2], [3, 4]))

In [2004]:
data.timestamps

[1, 2]

In [2005]:
data.values

[3, 4]

In [2006]:
data[0]

(1, 3)

In [2007]:

        data = InterfaceData([[1], [2]])
        data += ([2], [3])

In [2008]:
tuple(data)

((1, 2), (2, 3))

In [2009]:
data = InterfaceData([[1], [2]])
#data1 = data + InterfaceData(([], []))
data1 = data + (([], []))
data = InterfaceData([[4], [5]])

In [2010]:
tuple(data)

((4, 5),)

In [2011]:
tuple(data1)

((1, 2),)

In [2012]:
del data1

In [2013]:
data = InterfaceData(([1, 2], [3, 4]))

In [2014]:
data

In [2015]:
tuple(data)

((1, 3), (2, 4))

In [2016]:
data[0]

(1, 3)

In [2017]:
data[1]

(2, 4)

In [2018]:
len(InterfaceData())

0

In [2019]:
InterfaceData()

In [2020]:
tuple(InterfaceData())

()

In [2021]:
len(InterfaceData())

0

In [2022]:
len(InterfaceData(([1],[2])))

1

In [2023]:
timestamp, value = data[1]

In [2024]:
timestamp

2

In [2025]:
value

4

In [2026]:
data.append([[9,8,7,6],[34,45,56,67]])

In [2061]:
data[4:6]

([7, 6], [56, 67])

In [2028]:
([9], [34]) in data

True

In [2029]:
data in data + ([10], [34])

True

In [2030]:
data + ([10], [34]) in data

False

In [2031]:
([], []) in data

True

In [2032]:
data[::-1]

([6, 7, 8, 9, 2, 1], [67, 56, 45, 34, 4, 3])

In [2062]:
data = InterfaceData(([1, 2], [3, 4]))
assert data[0] == (1, 3), "Incorrect value"
assert data[1] == (2, 4), "Incorrect value"
assert data[::-1] == ([2, 1], [4, 3]), "Incorrect value"
data.append([[9, 8, 7, 6], [34, 45, 56, 67]])
assert data[4:6] == ([7, 6], [56, 67]), "Incorrect value"

In [2033]:
len(data)

6

In [2034]:
list(reversed(data))

[(6, 67), (7, 56), (8, 45), (9, 34), (2, 4), (1, 3)]

In [2035]:
cs = reversed(data)

In [2036]:
for timestamp, value in data:
    print(timestamp, value)

1 3
2 4
9 34
8 45
7 56
6 67


In [2037]:
for timestamp, value in reversed(data):
    print(timestamp, value)

6 67
7 56
8 45
9 34
2 4
1 3


In [2038]:
data + ([10], [34]) == data

False

In [2039]:
data == data

True

In [2040]:
data + ([10], [34]) > data

TypeError: '>' not supported between instances of 'InterfaceData' and 'InterfaceData'

In [2041]:

data == data

True

In [2042]:
data == ([10], [34])

False

In [2044]:
for sample in data:
    print(sample)
    print(InterfaceData(*sample) in data)

(1, 3)
True
(2, 4)
True
(9, 34)
True
(8, 45)
True
(7, 56)
True
(6, 67)
True


In [2047]:
data in reversed(data)

False

In [2050]:
data in list(reversed(data))

False

In [2051]:
list(reversed(data))

[(6, 67), (7, 56), (8, 45), (9, 34), (2, 4), (1, 3)]

In [2052]:
list(data)

[(1, 3), (2, 4), (9, 34), (8, 45), (7, 56), (6, 67)]

In [2056]:
list(data) in list(reversed(data))

False

In [2059]:
tuple(data) in data

ValueError: Samples passed to InterfaceData must be tuple([],[]) or timestamps, values or InterfaceData. Got (((1, 3), (2, 4), (9, 34), (8, 45), (7, 56), (6, 67)),)